In [12]:
'''
    Esse código baixa planilha de dados do TD
    e grava em um planilha (privada) do google docs
'''

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/spreadsheets']
jfile = 'carteira-328314-d38dcc8ee3e4.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(jfile, scope)
gc = gspread.authorize(credentials)

from datetime import datetime
import pandas as pd


In [13]:
titulo_planilha = 'Investimentos'
planilha = gc.open(titulo_planilha)
atualiza_saldos = True
pagina = planilha.worksheet('Renda Fixa')

In [14]:

from urllib.request import urlopen

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import json
url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'
response = urlopen(url)
data_json = json.loads(response.read())

In [15]:
nome , pr , pc , tr , tc = [],[],[],[],[]
td=pd.DataFrame()
for i in range(len(data_json["response"]["TrsrBdTradgList"])):
  nome.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['nm'])
  pr.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['untrRedVal'])
  pc.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['untrInvstmtVal'])
  tr.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['anulRedRate'])
  tc.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['anulInvstmtRate'])

td['título'] = nome
td['preço resgate'] = pr
td['preço compra'] = pc
td['taxa resgate'] = tr
td['taxa compra'] = tc

td['Ano'] = td['título'].str.extract(r'\b(\d{4})\b')
td = td.sort_values(by=['título','Ano'], ascending=True)
# td.drop(['Ano'],axis=1,inplace=True)

td

,título,preço resgate,preço compra,taxa resgate,taxa compra,Ano
23,Tesouro Educa+ 2026,3360.07,3374.78,6.4300,6.3100,2026
24,Tesouro Educa+ 2027,3160.07,3177.48,6.4200,6.3000,2027
26,Tesouro Educa+ 2028,2970.50,2990.20,6.4200,6.3000,2028
27,Tesouro Educa+ 2029,2792.33,2814.01,6.4200,6.3000,2029
28,Tesouro Educa+ 2030,2622.51,2645.84,6.4300,6.3100,2030
31,Tesouro Educa+ 2031,2462.67,2487.37,6.4400,6.3200,2031
32,Tesouro Educa+ 2032,2314.43,2340.27,6.4400,6.3200,2032
33,Tesouro Educa+ 2033,2177.45,2204.24,6.4300,6.3100,2033
34,Tesouro Educa+ 2034,2049.28,2076.81,6.4200,6.3000,2034
35,Tesouro Educa+ 2035,1931.29,1959.43,6.4000,6.2800,2035


In [16]:
td.drop(['Ano'],axis=1,inplace=True)

In [17]:
from datetime import date
today = date.today().strftime('%d/%m/%Y')

In [18]:
linha = int(input('linha da celula inicial (79)'))
pagina.update_cell(linha, 1,  today )

{'spreadsheetId': '1Mse441x9H8uByQkSTVngIKUmH5iBFwEuhGjo1LhqGE8',
 'updatedRange': "'Renda Fixa'!A79",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [19]:
pagina.update('a'+str(linha+1),[td.columns.values.tolist()] + td.values.tolist())

/tmp/ipykernel_44490/957776509.py:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  pagina.update('a'+str(linha+1),[td.columns.values.tolist()] + td.values.tolist())


{'spreadsheetId': '1Mse441x9H8uByQkSTVngIKUmH5iBFwEuhGjo1LhqGE8',
 'updatedRange': "'Renda Fixa'!A80:E137",
 'updatedRows': 58,
 'updatedColumns': 5,
 'updatedCells': 290}

In [20]:
meus_tits = []
for linha in range(3,11):
    meus_tits.append(pagina.cell(linha, 1).value)
meus_tits

['Tesouro IPCA+ 2035',
 'Tesouro IPCA+ 2045',
 'Tesouro IPCA+ com Juros Semestrais 2032',
 'Tesouro IPCA+ com Juros Semestrais 2035',
 'Tesouro IPCA+ com Juros Semestrais 2040',
 'Tesouro IPCA+ com Juros Semestrais 2055',
 'Tesouro Prefixado 2029',
 'Tesouro IPCA+ 2026']

In [21]:
meus = td[td['título'].isin(meus_tits)].copy()
meus['título']= meus['título'].str.replace('Tesouro ', '')
meus['título']= meus['título'].str.replace('com Juros Semestrais', 'Juros')
meus['título']= meus['título'].str.replace('20', '')

meus = meus.rename(columns={'título':str(today)})

meus

,23/06/2024,preço resgate,preço compra,taxa resgate,taxa compra
19,IPCA+ 26,3749.94,0.00,6.52,0.00
29,IPCA+ 35,2184.66,2211.51,6.44,6.32
42,IPCA+ 45,1172.18,1199.97,6.45,6.33
25,IPCA+ Juros 32,4276.57,0.00,6.44,0.00
30,IPCA+ Juros 35,4189.98,4228.19,6.44,6.32
36,IPCA+ Juros 40,4238.11,4287.47,6.38,6.26
49,IPCA+ Juros 55,4099.05,4164.44,6.43,6.31
11,Prefixado 29,598.34,0.00,12.09,0.00


In [22]:
pagina.update_cell(2, 13,  today )

{'spreadsheetId': '1Mse441x9H8uByQkSTVngIKUmH5iBFwEuhGjo1LhqGE8',
 'updatedRange': "'Renda Fixa'!M2",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [23]:
pagina.update('m2',[meus.columns.values.tolist()] + meus.values.tolist())

/tmp/ipykernel_44490/3471535783.py:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  pagina.update('m2',[meus.columns.values.tolist()] + meus.values.tolist())


{'spreadsheetId': '1Mse441x9H8uByQkSTVngIKUmH5iBFwEuhGjo1LhqGE8',
 'updatedRange': "'Renda Fixa'!M2:Q10",
 'updatedRows': 9,
 'updatedColumns': 5,
 'updatedCells': 45}